In [ ]:
!pip install requests beautifulsoup4

In [ ]:
!pip list

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

url_front = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%B4%EB%88%88&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=44&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start='

visitCount = 0

file = open('/content/gdrive/MyDrive/BigDataAnalaysis/naver-crawling.txt', 'w')

for pageIndex in range(1, 302, 10): # 실습 시간을 고려하여 302까지만 실행 (31페이지)
  pageUrl = '{0}{1}'.format(url_front, pageIndex)
  html = urlopen(pageUrl)
  bsObject = BeautifulSoup(html, 'html.parser')
  for link in bsObject.find_all('a', {'class':'news_tit'}):
    data = link.get('title') + '\n'
    file.write(data)

  visitCount += 1
  print('Read {0} page...'.format(visitCount))

  time.sleep(3)
file.close()
print('모든 데이터가 수집되었습니다.')

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Twitter
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# 수집한 데이터 title_list 리스트에 넣기
title_list = []

with open("/content/gdrive/MyDrive/BigDataAnalaysis/naver-crawling.txt", 'r', encoding='utf-8') as file :
    title_list = file.readlines()

# 형태소 분석하여 sentences_tag 리스트에 넣기
twitter = Twitter()
sentences_tag = []

for sentence in title_list :
    morph = twitter.pos(sentence)
    sentences_tag.append(morph)
    print(morph)
    print('-'*3)

print(sentences_tag)
print('\n'*3)

# 명사와 형용사만 구분하여 noun_adj_list 리스트에 넣기
noun_adj_list = []

for sentence1 in sentences_tag :
    for word, tag in sentence1 :
        if tag in ['Noun', 'Adjective'] :
            noun_adj_list.append(word)

# 형태소별 단어 카운트 및 상위 30개 단어 tags 리스트에 넣기
counts = Counter(noun_adj_list)
tags = counts.most_common(30)
print(tags)

# 한글 깨지는 문제 해결하기 위해 폰트 설정
wc = WordCloud(font_path='/content/gdrive/MyDrive/BigDataAnalaysis/NanumGothic.ttf', background_color='white', width=800, height=600)

# 워드 클라우드 생성
cloud = wc.generate_from_frequencies(dict(tags))
plt.figure(figsize=(10,8))
plt.axis('off')
plt.imshow(cloud)
plt.show()